### Quantum Computing

source: Quantum Computing For Computer Scientists

In [1]:
import cmath
import math
import numpy as np
from IPython.core.debugger import set_trace

#### Basic Operations

In [2]:
def adjoint(A):
    return A.getH()

def hermitian(A):
    return np.array_equal(adjoint(A),A)

def unitary(A):
    return np.array_equal(adjoint(A)@A,I(A.shape[0]))

def tensor(A,B):
    return np.kron(A,B)

def bra(ket):
    return np.array(list(map(lambda x:x.conjugate(),ket)))

#<a|b>
#probability of b transitioning to a
def transition_amplitude(a,b):
    return a@bra(b).reshape(-1,1)

def phase(c):
    return cmath.phase(c)

def l2_help(c):
    if isinstance(c,complex):
        return (c*c.conjugate()).real
    if isinstance(c,np.ndarray):
        if c.shape==(1,1):
            return l2_help(c[0,0])
        return sum(l2_help(i) for i in c)
    return c**2

def l2_norm(c):
    return l2_help(c)**(1/2)    

def normalize(c):
    return (1/l2_norm(c))*c

In [3]:
#2x2 (1-bit)
def Not():
    return np.matrix([[0,1],[1,0]])

#nxn
def I(n):
    return np.matrix([[1 if i==j else 0 for i in range(n)] for j in range(n)],dtype=complex)

#4x4 (2-bits)
def controlled_Not():
    r=I(4)
    r[2],r[3] = r[3].copy(),r[2].copy()
    return r


#8x8 (3-bit)
def Toffoli():
    r=I(8)
    r[6],r[7]=r[7],r[6]
    return r


#Pauli Matrices (More Quantum Gates)

def X():
    return Not()

def Y():
    return np.matrix([[0,complex(0,-1)],[complex(0,1),0]])

def Z():
    return np.matrix([[1,0],[0,-1]])

#some others
def S():
    return np.matrix([[1,0],[0,complex(0,1)]])

def T():
    return np.matrix([[1,0],[0,2**.5/2*complex(1,1)]])

def H():
    return 1/2**.5*(X()+Z())

def sqrt_Not():
    return 1/sqrt(2)*np.matrix([[1,-1],[1,1]])



In [4]:
#Measurement Operators

def position(dimension,n):
    return np.matrix([[1 if i==j==dimension else 0 for i in range(n)] for j in range(n)])
def momentum(delta):
    pass
def spin(direction):
    pass

In [9]:
#Quantum Systems


class OneParticleSystem:
    def __init__(self,n_states):
        self.n_states=n_states
        self.impose(0)
    def impose(self,state):
        self.state=np.array([1 if i==state else 0 for i in range(self.n_states)])
        self.imposed=True
    def evolve(self,A,steps):
        for i in range(steps):
            self.state=A@self.state
        return self.state
    def __str__(self):
        return str(self.state)

class System:
    def __init__(self,opSystems):
        self.system=opSystems
        self.n_particles=len(self.system)
        self.imposed=False
    def __init__(self,n_particles,n_states):
        self.n_particles=n_particles
        self.n_states=n_states
        self.state_len = n_particles**n_states
        self.system = [OneParticleSystem(self.n_states) for i in range(n_particles)]
        self.impose([0 for i in range(n_particles)])
    #cur_states is a list of states of length n_particles. each state is in [0,self.states-1]
    def impose(self,cur_states):
        self.imposed=True 
        self.system[0].impose(cur_states[0])
        self.state=self.system[0].state
        for i in range(1,self.n_particles):
            self.system[i].impose(cur_states[i])
            self.state=tensor(self.state,self.system[i].state)
    def evolve(self,A,steps=1):
        for i in range(steps):
            self.state=(A@self.state).reshape(-1,1)
        return self.state
    def entangled(self):
        tensored = self.state.reshape(*(self.system[i].n_states for i in range(self.n_particles)))
        print("tensored",tensored)
        return not (-.0001<np.linalg.det(tensored)<0.00001)
    def __str__(self):
        return str(self.state)+f" Entangled: {self.entangled()}"
    def predict(self,measurement):
        assert hermitian(measurement), "Measurement Operator must be Hermitian"
        ws,vs = np.linalg.eig(measurement)
        #set_trace()
        probabilities = [l2_norm(transition_amplitude(v,self.state))**2 for v in vs]
        #set_trace()
        mean = self.state.T@(measurement@self.state) #(xAx = <Ax,x>)
        return vs,probabilities,mean
    def measure(self,measurement):
        vs,probabilities,mean = self.predict(measurement)
        self.state=random.choice(vs,weights=probabilities)
        return self.state

class QuantumComputer(System):
    def __init__(self,n_qubits):
        System.__init__(self,n_qubits,2)

In [35]:
#plotting
import matplotlib.pyplot as plt

def plot_predict(vs,probabilities,mean):
    sVs = [str(v) for v in vs]
    plt.bar(sVs,probabilities)
    plt.xticks(rotation=30)

In [33]:
#Deutch's Algorithm
def Uf():
    r=I(4)
    r[1],r[2]=r[2].copy(),r[1].copy()
    return r

def deutchs():
    qc=QuantumComputer(2)
    qc.impose([0,1])
    print(qc)
    qc.evolve(tensor(H(),H()))
    print(qc)
    qc.evolve(controlled_Not())
    print(qc)
    qc.evolve(tensor(H(),I(2)))
    print(qc)
    plot_predict(*qc.predict(tensor(position(1,2),I(2))))

In [38]:
tensor(position(0,2),I(2))
#deutchs()


matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [8]:
%debug

> <ipython-input-5-905ff3b7f9b6>(52)predict()
     50         probabilities = [l2_norm(transition_amplitude(v,self.state))**2 for v in vs]
     51         #set_trace()
---> 52         mean = self.state@(measurement@self.state) #(xAx = <Ax,x>)
     53         return vs,probabilities,mean
     54     def measure(self,measurement):

ipdb> self.state
matrix([[ 2.29934717e-17+0.j],
        [-2.29934717e-17+0.j],
        [ 7.07106781e-01+0.j],
        [-7.07106781e-01+0.j]])
ipdb> self.state.shape
(4, 1)
ipdb> measurement.shape
(4, 4)
ipdb> (measurement@self.state).shape
(4, 1)
ipdb> exit
